In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.firefox.options import Options
import os
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
from selenium.webdriver.common.keys import Keys
import time

binary = FirefoxBinary('C:/Users/minif/Downloads/geckodriver-v0.23.0-win64/geckodriver.exe')

rootdir = "C:/Users/minif/OneDrive/Desktop/"
data = pd.read_excel(rootdir + "Options_Dates.xlsx")

start_dates_unix = data['UNIX Minus Week'].values.tolist()
end_dates_unix = data['UNIX Plus Week'].values.tolist()
tickers = data['Tickers'].values.tolist()

end_data = []

start = 1218
end = 1364
ticker_test = tickers[start:end]
start_dates_test = start_dates_unix[start:end]
end_dates_test = end_dates_unix[start:end]

final_data = {
    'Tickers': [],
    'Start Date': [],
    'Start Expiration Date': [],
    'Start Strike': [],
    'Start Bid Call': [],
    'Start Ask Call': [],
    'Start Bid IV Call': [],
    'Start Ask IV Call': [],
    'Start Volume Call': [],
    'Start Delta Call': [],
    'Start Open Interest Call': [],
    'Start Gamma Call': [],
    'Start Vega Call': [],
    'Start Theta Call': [],
    'Start Rho Call': [],
    'Start IV Change Call': [],
    'Start Market Change Call': [],
    'Start Bid Put': [],
    'Start Ask Put': [],
    'Start Bid IV Put': [],
    'Start Ask IV Put': [],
    'Start Volume Put': [],
    'Start Delta Put': [],
    'Start Open Interest Put': [],
    'Start Gamma Put': [],
    'Start Vega Put': [],
    'Start Theta Put': [],
    'Start Rho Put': [],
    'Start IV Change Put': [],
    'Start Market Change Put': [],
    'End Expiration Date': [],
    'End Strike': [],
    'End Bid Call': [],
    'End Ask Call': [],
    'End Bid IV Call': [],
    'End Ask IV Call': [],
    'End Volume Call': [],
    'End Delta Call': [],
    'End Open Interest Call': [],
    'End Gamma Call': [],
    'End Vega Call': [],
    'End Theta Call': [],
    'End Rho Call': [],
    'End IV Change Call': [],
    'End Market Change Call': [],
    'End Bid Put': [],
    'End Ask Put': [],
    'End Bid IV Put': [],
    'End Ask IV Put': [],
    'End Volume Put': [],
    'End Delta Put': [],
    'End Open Interest Put': [],
    'End Gamma Put': [],
    'End Vega Put': [],
    'End Theta Put': [],
    'End Rho Put': [],
    'End IV Change Put': [],
    'End Market Change Put': []
}

In [2]:
#open browser and login
delay = 10 # seconds
browser = webdriver.Firefox()
browser.get("https://marketchameleon.com/Overview/AAPL/OptionChain/")
try:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'site-header')))
except (TimeoutException, UnexpectedAlertPresentException) as e:
    myElem = WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'site-header')))
    print("Loading took too much time!")
    
browser.find_element_by_xpath('//*[@title="Log In"]').click()
    
try:
    WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.LINK_TEXT, '403 - Forbidden: Access is denied.')))
except TimeoutException:
    o = 0
    # do nothing and continue
WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.ID, 'UserName')))

username_box = browser.find_element_by_xpath('//*[@id="UserName"]')
password_box = browser.find_element_by_xpath('//*[@id="Password"]')

username_box.send_keys('johnny.saldana99@gmail.com')
password_box.send_keys('123456789')
password_box.send_keys(Keys.ENTER)
WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a.montage_datepicker_trigger')))

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="56914a95-2b5a-4f85-87a3-7cf9272e57b9", element="2e8a8e92-469a-42de-90d4-06d16c7ddf91")>

In [3]:
def check_value(val):
    if val == "":
        return "NaN"
    else:
        return val

In [4]:
def find_row(row,html, end_start, date, exp_date, ticker):
    # get the top strike price, bid_call, ask_call, bid_IV_call, ask_IV_call, bid_put, ask_put, bid_IV_put, ask_IV_put
    index_original = html.find('montage_atm_' + row)
    
    # strike
    index = html.find('montagelink', index_original)
    index = html.find('montagelink', index + 1)
    #the index of the strike price
    index_middle = index
    index = html.find('>', index)
    index_end = html.find('<',index)
    
    strike_price = html[index + 1: index_end]
    strike_price = check_value(strike_price)
    
    # del_call
    index = html.find('data-identity="cDelta"', index_original)
    index = html.find('>', index)
    index_end = html.find('<', index)
    del_call = html[index+1: index_end]
    del_call = check_value(del_call)
    
    # open_int_call
    index = html.find('cOpenInt', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    open_int_call = html[index+1: index_end]
    open_int_call = check_value(open_int_call)
    
    # gamma_call
    index = html.find('cGamma', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    gamma_call = html[index+1: index_end]
    gamma_call = check_value(gamma_call)
    
    # vega_call
    index = html.find('cVega', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    vega_call = html[index+1: index_end]
    vega_call = check_value(vega_call)
    
    # theta_call
    index = html.find('cTheta', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    theta_call = html[index+1: index_end]
    theta_call = check_value(theta_call)
    
    # rho_call
    index = html.find('cRho', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    rho_call = html[index+1: index_end]
    rho_call = check_value(rho_call)
    
    # IV_chg_call
    index = html.find('cIVChg', index)
    index = html.find('num', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    IV_chg_call = html[index+1: index_end]
    IV_chg_call = check_value(IV_chg_call)
    
    # mkt_chg_call
    index = html.find('cDelta', index)
    index = html.find('num', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    mkt_chg_call = html[index+1: index_end]
    mkt_chg_call = check_value(mkt_chg_call)
    
    # vol_call
    index = html.find('montagelink', index_original)
    index = html.find('>', index)
    index_end = html.find('<', index)
    vol_call = html[index + 1: index_end]
    vol_call = check_value(vol_call)
    
    # bid_IV_call
    index = html.find('volcell', index_original)
    index = html.find('>', index)
    index_end = html.find('<', index)
    bid_IV_call = html[index + 1: index_end]
    bid_IV_call = check_value(bid_IV_call)
    
    # bid_call
    index = html.find('pricecell', index_original)
    index = html.find('>', index)
    index_end = html.find('<', index)
    bid_call = html[index + 1: index_end]
    bid_call = check_value(bid_call)
    
    # ask_call
    index = html.find('pricecell', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    ask_call = html[index + 1: index_end]
    ask_call = check_value(ask_call)
    
    # ask_IV_call
    index = html.find('volcell', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    ask_IV_call = html[index + 1: index_end]
    ask_IV_call = check_value(ask_IV_call)
    
    # del_put
    index = html.find('data-identity="pDelta"', index_middle)
    index = html.find('>', index)
    index_end = html.find('<', index)
    del_put = html[index+1: index_end]
    del_put = check_value(del_put)
    
    # open_int_put
    index = html.find('pOpenInt', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    open_int_put = html[index+1: index_end]
    open_int_put = check_value(open_int_put)
    
    # gamma_put
    index = html.find('pGamma', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    gamma_put = html[index+1: index_end]
    gamma_put = check_value(gamma_put)
    
    # vega_put
    index = html.find('pVega', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    vega_put = html[index+1: index_end]
    vega_put = check_value(vega_put)
    
    # theta_put
    index = html.find('pTheta', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    theta_put = html[index+1: index_end]
    theta_put = check_value(theta_put)
    
    # rho_put
    index = html.find('pRho', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    rho_put = html[index+1: index_end]
    rho_put = check_value(rho_put)
    
    # IV_chg_put
    index = html.find('pIVChg', index)
    index = html.find('num', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    IV_chg_put = html[index+1: index_end]
    IV_chg_put = check_value(IV_chg_put)
    
    # mkt_chg_put
    index = html.find('pDelta', index)
    index = html.find('num', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    mkt_chg_put = html[index+1: index_end]
    mkt_chg_put = check_value(mkt_chg_put)
    
    # bid_IV_put
    index = html.find('volcell', index_middle)
    index = html.find('>', index)
    index_end = html.find('<', index)
    bid_IV_put = html[index + 1: index_end]
    bid_IV_put = check_value(bid_IV_put)
    
    # bid_put
    index = html.find('pricecell', index_middle)
    index = html.find('>', index)
    index_end = html.find('<', index)
    bid_put = html[index + 1: index_end]
    bid_put = check_value(bid_put)
    
    # ask_put
    index = html.find('pricecell', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    ask_put = html[index + 1: index_end]
    ask_put = check_value(ask_put)
    
    # ask_IV_put
    index = html.find('volcell', index)
    index = html.find('>', index)
    index_end = html.find('<', index)
    ask_IV_put = html[index + 1: index_end]
    ask_IV_put = check_value(ask_IV_put)
    
    # vol_put
    index = html.find('montagelink', index_middle)
    index = html.find('>', index)
    index_end = html.find('<', index)
    vol_put = html[index + 1: index_end]
    vol_put = check_value(vol_put)
    
    if end_start == 'start':
        final_data['Tickers'].append(ticker)
        final_data['Start Date'].append(date)
        final_data['Start Expiration Date'].append(exp_date)
        final_data['Start Strike'].append(strike_price)
        final_data['Start Bid Call'].append(bid_call)
        final_data['Start Ask Call'].append(ask_call)
        final_data['Start Bid IV Call'].append(bid_IV_call)
        final_data['Start Ask IV Call'].append(ask_IV_call)
        final_data['Start Volume Call'].append(vol_call)
        final_data['Start Delta Call'].append(del_call)
        final_data['Start Open Interest Call'].append(open_int_call)
        final_data['Start Gamma Call'].append(gamma_call)
        final_data['Start Vega Call'].append(vega_call)
        final_data['Start Theta Call'].append(theta_call)
        final_data['Start Rho Call'].append(rho_call)
        final_data['Start IV Change Call'].append(IV_chg_call)
        final_data['Start Market Change Call'].append(mkt_chg_call)
        final_data['Start Bid Put'].append(bid_put)
        final_data['Start Ask Put'].append(ask_put)
        final_data['Start Bid IV Put'].append(bid_IV_put)
        final_data['Start Ask IV Put'].append(ask_IV_put)
        final_data['Start Volume Put'].append(vol_put)
        final_data['Start Delta Put'].append(del_put)
        final_data['Start Open Interest Put'].append(open_int_put)
        final_data['Start Gamma Put'].append(gamma_put)
        final_data['Start Vega Put'].append(vega_put)
        final_data['Start Theta Put'].append(theta_put)
        final_data['Start Rho Put'].append(rho_put)
        final_data['Start IV Change Put'].append(IV_chg_put)
        final_data['Start Market Change Put'].append(mkt_chg_put)
    elif end_start == 'end':
        final_data['End Expiration Date'].append(exp_date)
        final_data['End Strike'].append(strike_price)
        final_data['End Bid Call'].append(bid_call)
        final_data['End Ask Call'].append(ask_call)
        final_data['End Bid IV Call'].append(bid_IV_call)
        final_data['End Ask IV Call'].append(ask_IV_call)
        final_data['End Volume Call'].append(vol_call)
        final_data['End Delta Call'].append(del_call)
        final_data['End Open Interest Call'].append(open_int_call)
        final_data['End Gamma Call'].append(gamma_call)
        final_data['End Vega Call'].append(vega_call)
        final_data['End Theta Call'].append(theta_call)
        final_data['End Rho Call'].append(rho_call)
        final_data['End IV Change Call'].append(IV_chg_call)
        final_data['End Market Change Call'].append(mkt_chg_call)
        final_data['End Bid Put'].append(bid_put)
        final_data['End Ask Put'].append(ask_put)
        final_data['End Bid IV Put'].append(bid_IV_put)
        final_data['End Ask IV Put'].append(ask_IV_put)
        final_data['End Volume Put'].append(vol_put)
        final_data['End Delta Put'].append(del_put)
        final_data['End Open Interest Put'].append(open_int_put)
        final_data['End Gamma Put'].append(gamma_put)
        final_data['End Vega Put'].append(vega_put)
        final_data['End Theta Put'].append(theta_put)
        final_data['End Rho Put'].append(rho_put)
        final_data['End IV Change Put'].append(IV_chg_put)
        final_data['End Market Change Put'].append(mkt_chg_put)

In [5]:
def find_rows(html, exp_date, start_end, a_date, ticker):
    find_row('top', html, start_end, a_date, exp_date, ticker)
    find_row('middle', html, start_end, a_date, exp_date, ticker)
    find_row('bottom', html, start_end, a_date, exp_date, ticker)

In [6]:
def split_date(date):
    year = datetime.utcfromtimestamp(date).strftime('%Y')
    month = datetime.utcfromtimestamp(date).strftime('%b')
    day = datetime.utcfromtimestamp(date).strftime('%d')
    day_week = datetime.utcfromtimestamp(date).strftime('%A')
    month_value = datetime.utcfromtimestamp(date).strftime('%m')
    month_value = str(int(month_value) - 1)
    
    if int(day) < 10:
        day = day[1:]
    
    return [year, month, day, day_week, month_value]

In [7]:
def download_data(ticker,year,month,day, start_end, date, month_value):
    browser.get("https://marketchameleon.com/Overview/" + ticker + "/OptionChain/")

    #need to click button to login
    #input login info and click another button to sign in
    #username: johnny.saldana99@gmail.com
    #password: 123456789
    #then we press the button to change the date to the one that we enter
    #then we click the download button
    #then we click 2nd expiration date
    #then we click the download button
    #then we click 3rd expiration date
    #then we click the download button
    #move on to next ticker
    
    try:
        WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.LINK_TEXT, 'It appears that ticker ' + ticker + ' is no longer actively trading.')))
        #browser.close()
        return 0
    except (TimeoutException, UnexpectedAlertPresentException) as e:
        pass
    
    try:
        WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a.montage_datepicker_trigger')))
    except TimeoutException:
        try:
            WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a.montage_datepicker_trigger')))
        except TimeoutException:
            #browser.close()
            return 0
    
    # get the right date as entered above
    date_btn = browser.find_element_by_xpath('//a[@class="montage_datepicker_trigger"]')
    WebDriverWait(browser, delay).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.montage_datepicker_trigger')))
    date_btn.click()
    
    try:
        WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'select.ui-datepicker-year')))
    except TimeoutException:
        date_btn = browser.find_element_by_xpath('//a[@class="montage_datepicker_trigger"]')
        date_btn.click()
        WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'select.ui-datepicker-year')))
        
    print(ticker, year, month, day)
    year_btn_holder = browser.find_element_by_xpath('//select[@class="ui-datepicker-year"]')
    year_btn = year_btn_holder.find_element_by_xpath('//option[@value="' + year + '"]')
    year_btn.click()
    WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//select[@class="ui-datepicker-month"]')))
    month_btn_holder = browser.find_element_by_xpath('//select[@class="ui-datepicker-month"]')
    month_btn = month_btn_holder.find_element_by_xpath('//option[text() = "' + month + '"]')
    month_btn.click()
    day_btn_holder = browser.find_element_by_xpath('//table[@class="ui-datepicker-calendar"]')
    day_btn = day_btn_holder.find_element_by_xpath('//a[text()="' + day + '"]/ancestor::td[@data-month="' + month_value + '"]')
    day_btn.click()
    
    WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//a[text()="' + day + '-' + month + '-' + year + '"]')))
    
    try:
        WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'tr.montage_atm_top')))
    except TimeoutException:
        try:
            WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//td[text() = "No data available in table"]')))
            print('found it')
            #browser.close()
            return 0
        except TimeoutException:
            print('found it - 2')
            #browser.close()
            return 0
        
    #print(html)
    try:
        try:
            el = browser.find_element_by_xpath('//tr[contains(@class, "odd")]')
        except StaleElementReferenceException:
            WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//tr[contains(@class, "odd"]')))
            el = browser.find_element_by_xpath('//tr[contains(@class, "odd")]')
        #WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//tr[contains(@class, "odd"]')))
        el = el.find_element_by_tag_name('td')
        try:
            date_el = el.find_element_by_tag_name('span')
        except StaleElementReferenceException:
            WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//tr[contains(@class, "odd"]')))
            date_el = el.find_element_by_tag_name('span')
        try:
            date_text = date_el.text
        except StaleElementReferenceException:
            WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//tr[contains(@class, "odd"]')))
            date_text = date_el.text
    except StaleElementReferenceException:
        try:
            el = browser.find_element_by_xpath('//tr[contains(@class, "odd"]')
        except StaleElementReferenceException:
            WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//tr[contains(@class, "odd"]')))
            el = browser.find_element_by_xpath('//tr[contains(@class, "odd"]')
        WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//tr[contains(@class, "odd"]/td')))
        el = el.find_element_by_tag_name('td')
        try:
            date_el = el.find_element_by_tag_name('span')
        except StaleElementReferenceException:
            WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//tr[contains(@class, "odd"]')))
            date_el = el.find_element_by_tag_name('span')
        try:
            date_text = date_el.text
        except StaleElementReferenceException:
            WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//tr[contains(@class, "odd"]')))
            date_text = date_el.text
    
    # get the html of the page
    html = browser.page_source
    find_rows(html, date_text, start_end, date, ticker)
    
    # click the second expiration date
    try:
        try:
            WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//tr[@class="even"]')))
        except TimeoutException:
            undo_previous(3,start_end)
            return 0
        el = browser.find_element_by_xpath('//tr[@class="even"]')
         
        el = el.find_element_by_tag_name('td')
        try:
            date_el = el.find_element_by_tag_name('span')
        except StaleElementReferenceException:
            WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//tr[@class="even"]')))
            date_el = el.find_element_by_tag_name('span')
        date_text = date_el.text
        el = browser.find_element_by_xpath('//tr[@class="even"]/td')
        el.click()
        try:
            WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'tr.montage_atm_top')))
        except TimeoutException:
            try:
                WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//td[text() = "No data available in table"]')))
                undo_previous(3, start_end)
                #browser.close()
                return 0
            except TimeoutException:
                undo_previous(3, start_end)
                #browser.close()
                return 0
    except StaleElementReferenceException:
        el = browser.find_element_by_xpath('//tr[@class="even"]')
        el = el.find_element_by_tag_name('td')
        try:
            date_el = el.find_element_by_tag_name('span')
        except StaleElementReferenceException:
            WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//tr[@class="even"]')))
            date_el = el.find_element_by_tag_name('span')
        date_text = date_el.text
        el = browser.find_element_by_xpath('//tr[@class="even"]/td')
        el.click()
        try:
            WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'tr.montage_atm_top')))
        except TimeoutException:
            try:
                WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//td[text() = "No data available in table"]')))
                undo_previous(3, start_end)
                #browser.close()
                return 0
            except TimeoutException:
                undo_previous(3, start_end)
                #browser.close()
                return 0
            
    html = browser.page_source
    find_rows(html, date_text, start_end, date, ticker)
    
    # click the third expiration date
    try:
        el = browser.find_element_by_xpath('//tr[@class="odd"][2]')
        el = el.find_element_by_tag_name('td')
        date_el = el.find_element_by_tag_name('span')
        date_text = date_el.text
        el.click()
        try:
            WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'tr.montage_atm_top')))
        except TimeoutException:
            try:
                WebDriverWait(browser, delay).until(EC.presence_of_element_located((By.XPATH, '//td[text() = "No data available in table"]')))
                
                undo_previous(6, start_end)
                #browser.close()
                return 0
            except TimeoutException:
                undo_previous(6, start_end)
                #browser.close()
                return 0
        html = browser.page_source
        find_rows(html, date_text, start_end, date, ticker)
    except NoSuchElementException:
        #browser.close()
        return 0
    
    #browser.close()
    return 5

In [1]:
def write_data_excel(data):
    df_output = pd.DataFrame(data)
    
    writer = pd.ExcelWriter('C:/Users/minif/OneDrive/Desktop/options_data' + str(start) + '_' + str(end) + '.xlsx')
    df_output.to_excel(writer,'Sheet1')
    writer.save()
    
    return df_output


NameError: name 'ticker_dic' is not defined

In [9]:
def undo_previous(num, start_end):
    if start_end == 'start':
        for i in range(num):
            final_data['Tickers'].pop()
            final_data['Start Date'].pop()
            final_data['Start Expiration Date'].pop()
            final_data['Start Strike'].pop()
            final_data['Start Bid Call'].pop()
            final_data['Start Ask Call'].pop()
            final_data['Start Bid IV Call'].pop()
            final_data['Start Ask IV Call'].pop()
            final_data['Start Volume Call'].pop()
            final_data['Start Delta Call'].pop()
            final_data['Start Open Interest Call'].pop()
            final_data['Start Gamma Call'].pop()
            final_data['Start Vega Call'].pop()
            final_data['Start Theta Call'].pop()
            final_data['Start Rho Call'].pop()
            final_data['Start IV Change Call'].pop()
            final_data['Start Market Change Call'].pop()
            final_data['Start Bid Put'].pop()
            final_data['Start Ask Put'].pop()
            final_data['Start Bid IV Put'].pop()
            final_data['Start Ask IV Put'].pop()
            final_data['Start Volume Put'].pop()
            final_data['Start Delta Put'].pop()
            final_data['Start Open Interest Put'].pop()
            final_data['Start Gamma Put'].pop()
            final_data['Start Vega Put'].pop()
            final_data['Start Theta Put'].pop()
            final_data['Start Rho Put'].pop()
            final_data['Start IV Change Put'].pop()
            final_data['Start Market Change Put'].pop()
    elif start_end == 'end':
        for i in range(num):
            final_data['End Expiration Date'].pop()
            final_data['End Strike'].pop()
            final_data['End Bid Call'].pop()
            final_data['End Ask Call'].pop()
            final_data['End Bid IV Call'].pop()
            final_data['End Ask IV Call'].pop()
            final_data['End Volume Call'].pop()
            final_data['End Delta Call'].pop()
            final_data['End Open Interest Call'].pop()
            final_data['End Gamma Call'].pop()
            final_data['End Vega Call'].pop()
            final_data['End Theta Call'].pop()
            final_data['End Rho Call'].pop()
            final_data['End IV Change Call'].pop()
            final_data['End Market Change Call'].pop()
            final_data['End Bid Put'].pop()
            final_data['End Ask Put'].pop()
            final_data['End Bid IV Put'].pop()
            final_data['End Ask IV Put'].pop()
            final_data['End Volume Put'].pop()
            final_data['End Delta Put'].pop()
            final_data['End Open Interest Put'].pop()
            final_data['End Gamma Put'].pop()
            final_data['End Vega Put'].pop()
            final_data['End Theta Put'].pop()
            final_data['End Rho Put'].pop()
            final_data['End IV Change Put'].pop()
            final_data['End Market Change Put'].pop()

In [10]:
'''day_weeks_start = []
day_weeks_end = []
for n in range(len(ticker_test)):
    day_weeks_start.append(split_date(start_dates_test[n])[3])
    day_weeks_end.append(split_date(end_dates_test[n])[3])
    '''

'day_weeks_start = []\nday_weeks_end = []\nfor n in range(len(ticker_test)):\n    day_weeks_start.append(split_date(start_dates_test[n])[3])\n    day_weeks_end.append(split_date(end_dates_test[n])[3])\n    '

In [11]:
for n in range(len(ticker_test)):
    print(n)
    actual_start_date = start_dates_test[n]
    actual_end_date = end_dates_test[n]
    start_date = split_date(start_dates_test[n])
    end_date = split_date(end_dates_test[n])
    
    r = download_data(ticker_test[n], start_date[0], start_date[1], start_date[2], 'start', actual_start_date, start_date[4])
    
    if r != 0:
        download_data(ticker_test[n], end_date[0], end_date[1], end_date[2], 'end', actual_end_date, end_date[4])
        discrepancy = len(final_data['Tickers']) - len(final_data['End Gamma Put'])
        if discrepancy != 0:
            undo_previous(discrepancy, 'start')
    
    print(len(final_data['Tickers']))
    print(len(final_data['End Gamma Put']))

0
PFE 2018 Jul 11
PFE 2018 Jul 31
9
9
1
LLY 2018 Jul 11
LLY 2018 Jul 31
18
18
2
JNJ 2018 Jul 11
JNJ 2018 Jul 31
27
27
3
27
27
4
LLY 2018 Jul 12
LLY 2018 Jul 31
36
36
5
AGN 2018 Jul 12
AGN 2018 Jul 31
45
45
6
AGIO 2018 Jul 13
AGIO 2018 Jul 31
54
54
7
GSK 2018 Jul 16
GSK 2018 Aug 2
63
63
8
XLRN 2018 Jul 16
XLRN 2018 Aug 2
72
72
9
RETA 2018 Jul 16
RETA 2018 Aug 2
81
81
10
CELG 2018 Jul 16
CELG 2018 Aug 2
90
90
11
BGNE 2018 Jul 16
BGNE 2018 Aug 2
99
99
12
ABBV 2018 Jul 17
ABBV 2018 Aug 3
108
108
13
NBIX 2018 Jul 17
NBIX 2018 Aug 3
117
117
14
CRMD 2018 Jul 18
CRMD 2018 Aug 7
126
126
15
126
126
16
GSK 2018 Jul 18
GSK 2018 Aug 7
135
135
17
MRK 2018 Jul 18
MRK 2018 Aug 7
144
144
18
BIIB 2018 Jul 18
BIIB 2018 Aug 7
153
153
19
ARRY 2018 Jul 19
ARRY 2018 Aug 7
162
162
20
ATNX 2018 Jul 19
ATNX 2018 Aug 7
171
171
21
AZN 2018 Jul 19
AZN 2018 Aug 7
180
180
22
TNXP 2018 Jul 20
TNXP 2018 Aug 7
189
189
23
DRRX 2018 Jul 20
DRRX 2018 Aug 7
198
198
24
INSY 2018 Jul 20
INSY 2018 Aug 7
207
207
25
PGNX 2018 J

In [12]:
#undo_previous(9, 'start')
write_data_excel(final_data)

,Tickers,Start Date,Start Expiration Date,Start Strike,Start Bid Call,Start Ask Call,Start Bid IV Call,Start Ask IV Call,Start Volume Call,Start Delta Call,...,End Ask IV Put,End Volume Put,End Delta Put,End Open Interest Put,End Gamma Put,End Vega Put,End Theta Put,End Rho Put,End IV Change Put,End Market Change Put
0,PFE,1531267200,13-Jul-18,36.50,0.73,0.85,11.8,28.0,"1,374",0.87,...,14.6,39.50,-0.41,15,0.65,0.02,-0.03,0.00,-11.4,-8.7
1,PFE,1531267200,13-Jul-18,37.00,0.30,0.31,12.3,12.6,435,0.71,...,14.6,40.00,-0.74,42,0.56,0.01,-0.02,0.00,-8.7,+0.9
2,PFE,1531267200,13-Jul-18,37.50,0.04,0.06,10.1,11.7,"1,061",0.23,...,17.2,40.50,-0.97,NaN,0.12,0.00,0.00,0.00,+0.9,+2.3
3,PFE,1531267200,20-Jul-18,36.50,0.82,0.97,13.2,21.0,7,0.76,...,13.5,39.50,-0.44,NaN,0.43,0.03,-0.01,0.00,-5.0,-4.5
4,PFE,1531267200,20-Jul-18,37.00,0.44,0.45,12.5,12.6,695,0.63,...,13.2,40.00,-0.66,2,0.43,0.03,-0.01,-0.01,-4.5,+2.1
5,PFE,1531267200,20-Jul-18,37.50,0.17,0.18,11.3,11.5,183,0.36,...,22.1,40.50,-0.84,NaN,0.29,0.02,-0.01,-0.01,+2.1,-6.6
6,PFE,1531267200,27-Jul-18,13,0.90,0.93,13.2,14.2,36.50,0.76,...,13.2,39.50,-0.44,NaN,0.34,0.04,-0.01,-0.01,-2.1,-2.7
7,PFE,1531267200,27-Jul-18,37.00,0.53,0.56,12.1,12.8,29,0.60,...,12.7,40.00,-0.62,197,0.35,0.03,-0.01,-0.01,-2.7,-2.9
8,PFE,1531267200,27-Jul-18,37.50,0.26,0.29,11.5,12.3,37,0.40,...,17.2,40.50,-0.76,373,0.27,0.03,-0.01,-0.01,-2.9,-3.9
9,LLY,1531267200,13-Jul-18,1,0.74,0.80,12.5,14.4,87.00,0.71,...,17.9,98.00,-0.31,68,0.19,0.04,-0.08,0.00,+0.7,+1.5


In [13]:
print(len(final_data['Tickers']))
print(len(final_data['End Theta Put']))

1098
1098
